In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# import scinter
sys.path.insert(0,'/path/to/Scinter3')
import scinter_import
import scinter_measurement
import scinter_data
import scinter_plot

plt.style.use('fast')

# some useful constants in SI units
degrees = np.pi/180.
mas = degrees/1000./3600.
hour = 3600.
year = 365.25*24.*hour
au = 149597870700. #m
pc = 648000./np.pi*au #m
v_c = 299792458.
minute = 60.
day = 24.*hour
mHz = 1.0e-3
kHz = 1.0e+3
MHz = 1.0e+6
GHz = 1.0e+9
mus = 1.0e-6
sqrtmus = np.sqrt(mus)
kms = 1000.

## Organizing your scintillation analysis

The next cell shows how to save a processed version of an observation such that it serves as a basis for further analysis whose results can be easily found and used in the future.\
Keep in mind that not all functions of Scinter3 work in ipython. Some functions not used here need to be executed in the terminal instead.

In [ ]:
path_data = "/path/to/your/data"
path_results = "/path/chosen/by/you/for/processed/data/and/results"

obsname = "label_your_observation"
filename = os.path.join(path_data,"name_of_your_data_file.npz")

"""
Add this class to scinter_import:

class from_npz(scinter_data.intensity):
    def __init__(self,data_path):
        # - load data
        data_npz = np.load(data_path)
        self.DS = data_npz["DS"]
        self.nu = data_npz["nu"]
        self.mjd = data_npz["mjd"]
        self.DS = self.DS/np.std(self.DS)
        self.t = (self.mjd-self.mjd[0])*day
        self.recalculate()
"""

DS = scinter_import.from_npz(filename)

"""
The next step is to save the data for easy access and an organized structure of the results.
This just creates another copy in this case, but the idea is that you change the format
of the data and/or crop, downsample, or rescale the data here for later use.
"""

# This path will contain all observations of this group, e.g. the same source.
storage = scinter_measurement.storage(path_results)
# This step creates the directory for the new observation.
obs = storage.add_obs(obsname)
# This step creates the data and info files for the new observation.
obs.log_DS(DS)
# Basic information about the observation can be found in computations/logfile.yaml
# Optional step to add this observation to a subgroup list.
# One observation can belong to multiple lists and by default is already added to the 'all' list.
list_tutorial = "tutorial"
storage.addtolist(list_tutorial,obsname)

In [ ]:
# Choose the storage of data and analysis products
storage = scinter_measurement.storage("/mnt/c/Users/RapSp/Desktop/MPIfR/Fundi_tutorial/B1508+55")

# choose an observation within that storage
obsname = "label_of_your_observation"
# name to be used as title in matplotlib; should not contain _
plotname = "name that is readable by matplotlib"

# load the observation
obs = scinter_measurement.measurement(storage.data_path,obsname)

## The intensity class

Accessible data products:
- DS  : time vs frequency array of intensity
- t   : array of time bins in seconds from beginning of observation
- mjd : array of mjd of each time bin
- nu  : array of frequency channels in Hz

Info computed from above:
- N_t  : number of time bins
- N_nu : number of frequency channels
- dt   : length of single time bin
- dnu  : length of single frequency channel
- timespan  : duration of whole observation
- bandwidth : total bandwidth
- nu0  : central frequency
- ...

Methods (all keywords are optional):
- crop(t_min=,t_max=,nu_min=,nu_max=) cuts out subsections of data from physical boundaries
- slice(i0_t=,i1_t=,i0_nu=,i1_nu=) cuts out subsections of data from bin numbers
- downsample(t_sampling=,nu_sampling=) reduces pixel number by averaging

In [ ]:
# load the dynamic spectrum
DS = scinter_data.intensity(obs.data_path)

## scinter_plot

Collection of functions to shorten calls of matplotlib:
- draw_canvas : creates a figure of given size in pixels, updates matplotlib's rcParams, and adjusts its subplots
- dynamic_spectrum : adds a pcolormesh to the given axis and provides default axis labels. Supports downsampling only for plotting.
- more inbuilt plot types, easy to add more within

Can be used independently of other parts of Scinter3 to ease plotting

In [ ]:
figure = scinter_plot.draw_canvas(plot_width = 1000,plot_height = 500, plot_bottom = 0.12, plot_left = 0.12, plot_right = 1.0, plot_wspace = 0.1, plot_top=0.95, plot_hspace = 0.1, textsize=18, labelsize=16)
ax = figure.add_subplot(1,1,1)
plot = scinter_plot.dynamic_spectrum(DS.t,DS.nu,DS.DS,ax,vmin=0.,vmax=7.,title=plotname,nu_sampling=4)
figure.colorbar(plot, ax=ax)
figure.savefig(os.path.join(obs.obs_path,"DynSpec.png"))

## logic of scinter_data

Each data tranformation is stored as its own class that includes:
- a method to create the data transformation from input data
- arrays of the result as well as arrays of physical coordinates
- ways to save and load the data

In particular, all scinter_data classes except for input data should have these optional keywords:
- data_path : path where to store the data
- file_name : specific file to be created or loaded within data_path, otherwise default will be used
- overwrite : if set to False, computation will only be done if specified file does not exist, default is True

In [ ]:
# compute secondary spectrum
# via FFT, saving the result, or loading it if already computed and overwrite==False
SS = scinter_data.SecSpec_FFT(DS,data_path=obs.data_path,overwrite=True)

# plot it
figure = scinter_plot.draw_canvas(plot_width = 800,plot_height = 600, plot_bottom = 0.1, plot_left = 0.1, plot_wspace = 0.1, plot_hspace = 0.1, textsize=16, labelsize=12)
ax = figure.add_subplot(1,1,1)
plot = scinter_plot.secondary_spectrum(SS.fD,SS.tau,SS.SS,ax,fD_sampling=1,tau_sampling=8,fD_min=None,fD_max=None,tau_min=None,tau_max=None,vmin=None,vmax=None)
figure.colorbar(plot, ax=ax)
figure.savefig(os.path.join(obs.obs_path,"SecSpec_FFT.png"))

## Tracking results

`obs.enter_result(key,value)` saves a result in the logfile of the observation. From there, it can later be used to look it up or to use it for further analysis. A result can be read with `obs.results[key]`.

In [ ]:
# compute the autocorrelation function
ACF = scinter_data.ACF_DS(DS,data_path=obs.data_path,overwrite=False)

# fit the scintillation scales
nuscint,tscint,modindex,model_ACF,ccorr_t,ccorr_nu,t_model_ACF,nu_model_ACF = ACF.fit2D()

# save fit results
obs.enter_result('tscint',tscint)
obs.enter_result('nuscint',nuscint)

# plot diagnostic plot
figure = scinter_plot.draw_canvas(plot_width = 800,plot_height = 600, plot_bottom = 0.1, plot_left = 0.05, plot_top=0.9,plot_right=0.9, plot_wspace = 0.1, plot_hspace = 0.1, textsize=16, labelsize=12)
scinter_plot.scintscales(ACF.t_shift,ACF.nu_shift,ACF.ACF,ccorr_t,t_model_ACF,ccorr_nu,nu_model_ACF,figure,nu_sampling=1,t_min=-5*tscint/minute,t_max=5*tscint/minute,nu_min=-5*nuscint/MHz,nu_max=5*nuscint/MHz,cmap="viridis",corr_max=1.2*modindex**2)
plt.text(0.8, 0.75, "source name\n"+"{:.0f}".format(DS.nu0/MHz)+" MHz\n$t_{\\mathrm{ISS}}="+"{:.1f}".format(tscint)+"$ s\n$\\nu_{\\mathrm{ISS}}="+"{:.3f}".format(nuscint/kHz)+"$ kHz", transform=plt.gcf().transFigure)
figure.savefig(os.path.join(obs.obs_path,"scintscales.png"))

## Analyzing a series of observations

Lists of observation names can be obtained via `storage.obs_lists[name_of_list]`, which is useful to loop over them to apply the same analysis to many observations. Results saved in the logfiles can be retrieved as a numpy array with `storage.get_array(name_of_list,key)`\
Creating an observation with an existing name will not delete the results already saved under this name but only replace the dynamic spectrum!

In [ ]:
# import a whole directory of multiple data files
data_files = os.listdir(path_data)
for filename in data_files:
    # create obsname from filename; here omitting some letters
    obsname = filename[:3]+filename[9:18]
    # copy all data into the storage
    filename = os.path.join(path_data,filename)
    DS = scinter_import.from_npz(filename)
    obs = storage.add_obs(obsname)
    obs.log_DS(DS)
    storage.addtolist(list_tutorial,obsname)

In [ ]:
obs_list = storage.obs_lists[list_tutorial]

# compute scintillation scales for all observations in the list
for obsname in obs_list:
    obs = scinter_measurement.measurement(storage.data_path,obsname)
    DS = scinter_data.intensity(obs.data_path)
    ACF = scinter_data.ACF_DS(DS,data_path=obs.data_path,overwrite=False)
    nuscint,tscint,modindex,model_ACF,ccorr_t,ccorr_nu,t_model_ACF,nu_model_ACF = ACF.fit2D()
    obs.enter_result('tscint',tscint)
    obs.enter_result('nuscint',nuscint)

In [ ]:
# plot the variation of the scales over time

arr_mjd = storage.get_array(list_tutorial,"mjd0")
arr_nuscint = storage.get_array(list_tutorial,"nuscint")
arr_tscint = storage.get_array(list_tutorial,"tscint")

figure = scinter_plot.draw_canvas(plot_width = 800,plot_height = 800, plot_bottom = 0.1, plot_left = 0.1, plot_wspace = 0.1, plot_hspace = 0.1, textsize=16, labelsize=12)
ax1 = figure.add_subplot(2,1,1)
ax2 = figure.add_subplot(2,1,2)
ax1.plot(arr_mjd,arr_nuscint/kHz,marker='o',linestyle='')
ax2.plot(arr_mjd,arr_tscint,marker='o',linestyle='')
ax2.set_xlabel("MJD")
ax1.set_ylabel(r"$\nu_{\rm ISS}$ [kHz]")
ax2.set_ylabel(r"$t_{\rm ISS}$ [s]")
figure.savefig(os.path.join(storage.data_path,"scintscales_{0}.png".format(list_tutorial)))